In [5]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [8]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all elements that contain article information
article = soup.find('li', class_='slide')
news_title = article.find('div', class_="content_title").text
news_paragraph = article.find('div', class_="article_teaser_body").text
link = article.a['href']
news_article_link = 'https://mars.nasa.gov/' + link

print(news_title)
print(news_paragraph)
print(news_article_link)

NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/


Visit the url for JPL Featured Space Image here.

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

Make sure to find the image url to the full size .jpg image.

Make sure to save a complete url string for this image.

In [28]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Navigate to page with image info
browser.click_link_by_partial_text('FULL IMAGE')
browser.is_element_present_by_text("more info", wait_time=1)
more_info = browser.find_link_by_partial_text("more info")
more_info.click()

# Scrape image info
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image = soup.find('figure', class_='lede')
link = image.a['href']
featured_image_url = 'https://www.jpl.nasa.gov' + link
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17440_hires.jpg


Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [5]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

# Scrape weather report tweet 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather = soup.find('p', class_='tweet-text').text
print(mars_weather)


Happy Landiversary to MSL. In 7 years @MarsCuriosity has traveled 23 km (14 mi) and recorded air temps ranging from  -90ºC / -130ºF to 9ºC / 48.2ºF.  Its cold on Mars but ~9% of the Sols reported temps above freezing.pic.twitter.com/fcmnv4CmvY


Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

Use Pandas to convert the data to a HTML table string.

In [21]:
# URL of page to be scraped
url = 'https://space-facts.com/mars/'
browser.visit(url)

# # Scrape planet profile table 
table = pd.read_html(url)
# print(table)

# Using pandas to convert to DataFrame
mars_facts_df = pd.DataFrame(table[1])
mars_facts_df.columns=['Description','Value']
display(mars_facts_df)

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [ ]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)